In [74]:
from espn_api.football import League
import pandas as pd
# Init

#fan's - 42936131
#mine - 829963546
league_id = 829963546
year = 2022
league = League(league_id=league_id, year=year)



## Lets get the draft picks from each team

In [75]:
df = pd.DataFrame()

teams = []
players = []
id = []
round_num = []
round_pick = []


for pick in league.draft:
    teams.append(pick.team.team_name)
    players.append(pick.playerName)
    round_num.append(pick.round_num)
    round_pick.append(pick.round_pick)
    id.append(pick.playerId)
    
df['Team'] = teams
df['Player'] = players
df['ID'] = id
df['RoundPick'] = round_pick
df['RoundNum'] = round_num
df.to_csv('Data/team_players.csv')
df.head(10)

,Team,Player,ID,RoundPick,RoundNum
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1
1,Diggy Wit it,Christian McCaffrey,3117251,2,1
2,My Team A Bust,Austin Ekeler,3068267,3,1
3,Eugene Tweakers,Cooper Kupp,2977187,4,1
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1
5,Gorilla Guys,Derrick Henry,3043078,6,1
6,I dont know what im doing,Leonard Fournette,3115364,7,1
7,Cougar Town Zach Wilson,Deebo Samuel,3126486,8,1
8,No Nyugen Will Win,Najee Harris,4241457,9,1
9,Just-in-N-out Drive thru,Justin Jefferson,4262921,10,1


# Get projected points

In [76]:
players = []
expected = []
actual = []
rank = []
nfl_team = []
for team in league.teams:
    for player in team.roster:
        players.append(player.name)
        expected.append(player.projected_total_points)
        actual.append(player.total_points)
        rank.append(player.posRank)
        nfl_team.append(player.proTeam)

points_df = pd.DataFrame()
points_df['Player'] = players
points_df['Expected_Points'] = expected
points_df['Actual_Points'] = actual
points_df['Rank'] = rank
points_df['NFLTeam'] = nfl_team



In [77]:
points_df['Difference'] = points_df['Actual_Points'] - points_df['Expected_Points']

In [78]:
points_df = points_df.sort_values('Difference', ascending = False)
points_df.to_csv('Data/player_differences.csv')

In [79]:
points_df.groupby('Team').mean().sort_values('Difference', ascending=False).to_csv('Data/grouped_average_differences.csv')
points_df.groupby('Team').mean().sort_values('Difference', ascending=False)

KeyError: 'Team'

# Export the CSV

In [80]:
points_df.to_csv('Data/player_points.csv')
points_df.head(10)

,Player,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
145,Jimmy Garoppolo,14.84,149.26,19,SF,134.42
162,Kenyan Drake,13.48,79.20,41,BAL,65.72
140,Jeff Wilson Jr.,50.21,112.40,23,MIA,62.19
95,Geno Smith,148.82,182.76,7,SEA,33.94
28,Marcus Mariota,141.19,172.32,9,ATL,31.13
76,Jamaal Williams,123.09,153.50,13,DET,30.41
163,Eno Benjamin,64.37,86.30,37,HOU,21.93
141,Patriots D/ST,90.39,111.00,1,NE,20.61
94,Jets D/ST,70.84,91.00,4,NYJ,20.16
160,Josh Reynolds,58.02,73.70,61,DET,15.68


In [81]:
combined_df = pd.merge(df, points_df, on = 'Player')
combined_df.head(10)

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99
5,Gorilla Guys,Derrick Henry,3043078,6,1,264.47,203.06,2,TEN,-61.41
6,I dont know what im doing,Leonard Fournette,3115364,7,1,258.74,154.70,12,TB,-104.04
7,Cougar Town Zach Wilson,Deebo Samuel,3126486,8,1,261.46,130.00,21,SF,-131.46
8,No Nyugen Will Win,Najee Harris,4241457,9,1,265.48,124.96,20,PIT,-140.52
9,Just-in-N-out Drive thru,Justin Jefferson,4262921,10,1,287.80,215.52,4,MIN,-72.28


In [82]:
combined_df.to_csv('Data/total.csv')

In [83]:
combined_df

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-204.99
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-108.18
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-54.63
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-111.51
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-101.99
...,...,...,...,...,...,...,...,...,...,...
118,My Team A Bust,Tyler Bass,3917232,3,15,131.58,100.00,1,BUF,-31.58
119,Eugene Tweakers,Cowboys D/ST,-16006,4,15,106.49,109.00,2,DAL,2.51
120,Couv Reigning Champ,Harrison Butker,3055899,5,15,145.15,47.00,29,KC,-98.15
121,Cougar Town Zach Wilson,Brian Robinson Jr.,4241474,8,15,102.87,52.90,52,WSH,-49.97


In [84]:
#normalize
combined_df['Difference'] = combined_df['Difference'] /combined_df['Difference'].abs().max()
  

In [85]:
combined_df

,Team,Player,ID,RoundPick,RoundNum,Expected_Points,Actual_Points,Rank,NFLTeam,Difference
0,P-Town CASH KEVIN,Jonathan Taylor,4242335,1,1,316.99,112.00,24,IND,-0.912933
1,Diggy Wit it,Christian McCaffrey,3117251,2,1,309.94,201.76,3,SF,-0.481785
2,My Team A Bust,Austin Ekeler,3068267,3,1,284.73,230.10,1,LAC,-0.243297
3,Eugene Tweakers,Cooper Kupp,2977187,4,1,312.91,201.40,5,LAR,-0.496615
4,Couv Reigning Champ,Dalvin Cook,3116593,5,1,258.49,156.50,11,MIN,-0.454218
...,...,...,...,...,...,...,...,...,...,...
118,My Team A Bust,Tyler Bass,3917232,3,15,131.58,100.00,1,BUF,-0.140643
119,Eugene Tweakers,Cowboys D/ST,-16006,4,15,106.49,109.00,2,DAL,0.011178
120,Couv Reigning Champ,Harrison Butker,3055899,5,15,145.15,47.00,29,KC,-0.437116
121,Cougar Town Zach Wilson,Brian Robinson Jr.,4241474,8,15,102.87,52.90,52,WSH,-0.222544
